<br>

###   <font color="Coral">hw7pr2digits_modeler</font>
+ digits clasification -- and regression -- via NNets

Feel free to re-use your previous digits_cleaned.csv file 


### First, use the iris example to create a digit-predicting NNet

This is similar to the past two digits challenges, hw5 and hw6

**However**, because we're using NNets, you'll need to
+ create TEN categorical variables. You can use just one ``get_dummies`` pandas call
+ use a SCALER to make sure the network can, in a fair way, "hear" all of the inputs 

In [3]:
#
# coding cell to make sure Python is happy...

14*3

42

In [4]:
# libraries...
import numpy as np      # numpy is Python's "array" library
import pandas as pd     # Pandas is Python's "data" library ("dataframe" == spreadsheet)

In [5]:
# let's read in our digits data...
# 
#digits_cleaned.csv should be in this folder
# 
filename = 'digits_cleaned.csv' # neighborhoods
df_tidy = pd.read_csv(filename)      # encoding = "utf-8", "latin1"
print(f"{filename} : file read into a pandas dataframe.")

digits_cleaned.csv : file read into a pandas dataframe.


In [6]:
#
# different version vary on how to see all rows (adapt to suit your system!)
#
print(f"df_tidy.shape is {df_tidy.shape}\n")
df_tidy.info()  # prints column information
df_tidy

df_tidy.shape is (1768, 65)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1768 entries, 0 to 1767
Data columns (total 65 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   pix0          1768 non-null   int64
 1   pix1          1768 non-null   int64
 2   pix2          1768 non-null   int64
 3   pix3          1768 non-null   int64
 4   pix4          1768 non-null   int64
 5   pix5          1768 non-null   int64
 6   pix6          1768 non-null   int64
 7   pix7          1768 non-null   int64
 8   pix8          1768 non-null   int64
 9   pix9          1768 non-null   int64
 10  pix10         1768 non-null   int64
 11  pix11         1768 non-null   int64
 12  pix12         1768 non-null   int64
 13  pix13         1768 non-null   int64
 14  pix14         1768 non-null   int64
 15  pix15         1768 non-null   int64
 16  pix16         1768 non-null   int64
 17  pix17         1768 non-null   int64
 18  pix18         1768 non-null   int64
 19

,pix0,pix1,pix2,pix3,pix4,pix5,pix6,pix7,pix8,pix9,...,pix55,pix56,pix57,pix58,pix59,pix60,pix61,pix62,pix63,actual_digit
0,0,0,9,14,8,1,0,0,0,0,...,0,0,0,11,16,15,11,1,0,8
1,0,0,11,12,0,0,0,0,0,2,...,0,0,0,9,12,13,3,0,0,9
2,0,0,1,9,15,11,0,0,0,0,...,0,0,0,1,10,13,3,0,0,0
3,0,0,0,0,14,13,1,0,0,0,...,0,0,0,0,1,13,16,1,0,1
4,0,0,5,12,1,0,0,0,0,0,...,2,0,0,3,11,8,13,12,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1763,0,0,4,10,13,6,0,0,0,1,...,0,0,0,2,14,15,9,0,0,9
1764,0,0,6,16,13,11,1,0,0,0,...,0,0,0,6,16,14,6,0,0,0
1765,0,0,1,11,15,1,0,0,0,0,...,0,0,0,2,9,13,6,0,0,8
1766,0,0,2,10,7,0,0,0,0,0,...,0,0,0,5,12,16,12,0,0,9


In [7]:
#
# once we have all the columns we want, let's create an index of their names...

#
# Let's make sure we have all of our helpful variables in one place 
#       To be adapted if we drop/add more columns...
#

#
# let's keep our column names in variables, for reference
#
COLUMNS = df_tidy.columns            # "list" of columns
print(f"COLUMNS is {COLUMNS}\n")  
  # It's a "pandas" list, called an Index
  # use it just as a Python list of strings:
print(f"COLUMNS[0] is {COLUMNS[0]}\n")

# let's create a dictionary to look up any column index by name
COL_INDEX = {}
for i, name in enumerate(COLUMNS):
    COL_INDEX[name] = i  # using the name (as key), look up the value (i)
print(f"COL_INDEX is {COL_INDEX}\n\n")

#
# Feature names!
#
FEATURES = COLUMNS[0:64]

#
# and our "species" names
#

# all of scikit-learn's ML routines need numbers, not strings
#   ... even for categories/classifications (like species!)
#   so, we will convert the flower-species to numbers:

SPECIES = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']   # int to str

COLUMNS is Index(['pix0', 'pix1', 'pix2', 'pix3', 'pix4', 'pix5', 'pix6', 'pix7', 'pix8',
       'pix9', 'pix10', 'pix11', 'pix12', 'pix13', 'pix14', 'pix15', 'pix16',
       'pix17', 'pix18', 'pix19', 'pix20', 'pix21', 'pix22', 'pix23', 'pix24',
       'pix25', 'pix26', 'pix27', 'pix28', 'pix29', 'pix30', 'pix31', 'pix32',
       'pix33', 'pix34', 'pix35', 'pix36', 'pix37', 'pix38', 'pix39', 'pix40',
       'pix41', 'pix42', 'pix43', 'pix44', 'pix45', 'pix46', 'pix47', 'pix48',
       'pix49', 'pix50', 'pix51', 'pix52', 'pix53', 'pix54', 'pix55', 'pix56',
       'pix57', 'pix58', 'pix59', 'pix60', 'pix61', 'pix62', 'pix63',
       'actual_digit'],
      dtype='object')

COLUMNS[0] is pix0

COL_INDEX is {'pix0': 0, 'pix1': 1, 'pix2': 2, 'pix3': 3, 'pix4': 4, 'pix5': 5, 'pix6': 6, 'pix7': 7, 'pix8': 8, 'pix9': 9, 'pix10': 10, 'pix11': 11, 'pix12': 12, 'pix13': 13, 'pix14': 14, 'pix15': 15, 'pix16': 16, 'pix17': 17, 'pix18': 18, 'pix19': 19, 'pix20': 20, 'pix21': 21, 'pix22': 22, 'pix23'

In [8]:
#
# let's convert our dataframe to a numpy array, named A
#
A = df_tidy.to_numpy()   
A = A.astype('float64')    # many types:  www.tutorialspoint.com/numpy/numpy_data_types.htm
print(A[:,:])

# print(A)  # the five rows above is probably enough...  

[[ 0.  0.  9. ...  1.  0.  8.]
 [ 0.  0. 11. ...  0.  0.  9.]
 [ 0.  0.  1. ...  0.  0.  0.]
 ...
 [ 0.  0.  1. ...  0.  0.  8.]
 [ 0.  0.  2. ...  0.  0.  9.]
 [ 0.  0. 10. ...  1.  0.  8.]]


In [9]:
#
# nice to have NUM_ROWS and NUM_COLS around
#
NUM_ROWS, NUM_COLS = A.shape
print(f"\nThe dataset has {NUM_ROWS} rows and {NUM_COLS} cols")


The dataset has 1768 rows and 65 cols


In [10]:
#
# let's make sure it's all floating-point (here, it already is, but in other datasets it might not be)
#
A = A.astype('float64')  # so many:  www.tutorialspoint.com/numpy/numpy_data_types.htm
print(A)

[[ 0.  0.  9. ...  1.  0.  8.]
 [ 0.  0. 11. ...  0.  0.  9.]
 [ 0.  0.  1. ...  0.  0.  0.]
 ...
 [ 0.  0.  1. ...  0.  0.  8.]
 [ 0.  0.  2. ...  0.  0.  9.]
 [ 0.  0. 10. ...  1.  0.  8.]]


In [11]:
# let's use all our variables, to reinforce that we have
# (1) their names...
# (2) access and control over each...

# choose a row index, n:
n = 42
print(f"digit #{n} is {A[n]}")

for i in range(len(COLUMNS)):
    colname = COLUMNS[i]
    value = A[n][i]
    print(f"  Its {colname} is {value}")

digit_index = COL_INDEX['actual_digit']
digit_num = int(round(A[n][digit_index]))
print(f"  Its digit is {digit_num}")

digit #42 is [ 0.  0.  0. 10. 11.  0.  0.  0.  0.  0.  9. 16.  6.  0.  0.  0.  0.  0.
 15. 13.  0.  0.  0.  0.  0.  0. 14. 10.  0.  0.  0.  0.  0.  1. 15. 12.
  8.  2.  0.  0.  0.  0. 12. 16. 16. 16. 10.  1.  0.  0.  7. 16. 12. 12.
 16.  4.  0.  0.  0.  9. 15. 12.  5.  0.  6.]
  Its pix0 is 0.0
  Its pix1 is 0.0
  Its pix2 is 0.0
  Its pix3 is 10.0
  Its pix4 is 11.0
  Its pix5 is 0.0
  Its pix6 is 0.0
  Its pix7 is 0.0
  Its pix8 is 0.0
  Its pix9 is 0.0
  Its pix10 is 9.0
  Its pix11 is 16.0
  Its pix12 is 6.0
  Its pix13 is 0.0
  Its pix14 is 0.0
  Its pix15 is 0.0
  Its pix16 is 0.0
  Its pix17 is 0.0
  Its pix18 is 15.0
  Its pix19 is 13.0
  Its pix20 is 0.0
  Its pix21 is 0.0
  Its pix22 is 0.0
  Its pix23 is 0.0
  Its pix24 is 0.0
  Its pix25 is 0.0
  Its pix26 is 14.0
  Its pix27 is 10.0
  Its pix28 is 0.0
  Its pix29 is 0.0
  Its pix30 is 0.0
  Its pix31 is 0.0
  Its pix32 is 0.0
  Its pix33 is 1.0
  Its pix34 is 15.0
  Its pix35 is 12.0
  Its pix36 is 8.0
  Its pix37 is 2.0
 

In [12]:
print("+++ Start of data definitions +++\n")

#
# we could do this at the data-frame level, too!
#

X_all = A[:,0:64]   # X (features) ... is all rows, columns 0-63
y_all = A[:,64]    # y (labels) ... is all rows, column 64

print(f"y_all (just the labels/species, first few rows) are \n {y_all[0:5]}")
print()
print(f"X_all (just the features, first few rows) are \n {X_all[0:5]}")

+++ Start of data definitions +++

y_all (just the labels/species, first few rows) are 
 [8. 9. 0. 1. 2.]

X_all (just the features, first few rows) are 
 [[ 0.  0.  9. 14.  8.  1.  0.  0.  0.  0. 12. 14. 14. 12.  0.  0.  0.  0.
   9. 10.  0. 15.  4.  0.  0.  0.  3. 16. 12. 14.  2.  0.  0.  0.  4. 16.
  16.  2.  0.  0.  0.  3. 16.  8. 10. 13.  2.  0.  0.  1. 15.  1.  3. 16.
   8.  0.  0.  0. 11. 16. 15. 11.  1.  0.]
 [ 0.  0. 11. 12.  0.  0.  0.  0.  0.  2. 16. 16. 16. 13.  0.  0.  0.  3.
  16. 12. 10. 14.  0.  0.  0.  1. 16.  1. 12. 15.  0.  0.  0.  0. 13. 16.
   9. 15.  2.  0.  0.  0.  0.  3.  0.  9. 11.  0.  0.  0.  0.  0.  9. 15.
   4.  0.  0.  0.  9. 12. 13.  3.  0.  0.]
 [ 0.  0.  1.  9. 15. 11.  0.  0.  0.  0. 11. 16.  8. 14.  6.  0.  0.  2.
  16. 10.  0.  9.  9.  0.  0.  1. 16.  4.  0.  8.  8.  0.  0.  4. 16.  4.
   0.  8.  8.  0.  0.  1. 16.  5.  1. 11.  3.  0.  0.  0. 12. 12. 10. 10.
   0.  0.  0.  0.  1. 10. 13.  3.  0.  0.]
 [ 0.  0.  0.  0. 14. 13.  1.  0.  0.  0.  0.  5. 

In [13]:
#
# we can scramble the data, to remove (potential) dependence on its ordering: 
# 
indices = np.random.permutation(len(y_all))  # indices is a permutation-list

# we scramble both X and y, necessarily with the same permutation
X_all = X_all[indices]              # we apply the _same_ permutation to each!
y_all = y_all[indices]              # again...
print(f"The scrambled labels/species are \n {y_all[0:5]}")
print()
print(f"The corresponding data rows are \n {X_all[0:5]}")

The scrambled labels/species are 
 [2. 6. 6. 9. 1.]

The corresponding data rows are 
 [[ 0.  0. 10. 16.  8.  0.  0.  0.  0.  4. 16. 13. 16.  3.  0.  0.  0.  0.
  12.  1. 11.  6.  0.  0.  0.  0.  0.  0. 12.  8.  0.  0.  0.  0.  0.  0.
  14.  5.  0.  0.  0.  0.  0.  7. 16.  6.  4.  0.  0.  0.  5. 16. 16. 16.
  16.  4.  0.  0. 11. 15.  9.  8.  6.  0.]
 [ 0.  0.  2. 16.  8.  0.  0.  0.  0.  0.  8. 16.  6.  0.  0.  0.  0.  0.
  15. 10.  0.  0.  0.  0.  0.  4. 16.  2.  0.  0.  0.  0.  0.  8. 16. 16.
  16. 14.  2.  0.  0.  8. 16.  7.  4. 16.  8.  0.  0.  1. 16.  9.  6. 16.
   4.  0.  0.  0.  3. 12. 16. 12.  0.  0.]
 [ 0.  0.  0. 12.  4.  0.  0.  0.  0.  0.  6. 14.  1.  0.  0.  0.  0.  0.
  14.  2.  0.  0.  0.  0.  0.  2. 14.  1.  4.  2.  0.  0.  0.  4. 16. 15.
  12. 15.  5.  0.  0.  3. 16.  6.  0.  5. 11.  0.  0.  0.  9. 11.  4. 13.
   5.  0.  0.  0.  1. 11. 16.  9.  0.  0.]
 [ 0.  0.  9. 16. 11.  1.  0.  0.  0.  5. 16. 10. 16.  9.  0.  0.  0.  6.
  14.  1.  9. 15.  0.  0.  0.  1. 15.  6. 11

In [14]:
#
# We next separate into test data and training data ... 
#    + We will train on the training data...
#    + We will _not_ look at the testing data to build the model
#
# Then, afterward, we will test on the testing data -- and see how well we do!
#

#
# a common convention:  train on 80%, test on 20%    Let's define the TEST_PERCENT
#

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2)

print(f"training with {len(y_train)} rows;  testing with {len(y_test)} rows\n" )

print(f"+++ Testing +++   Held-out data... (testing data: {len(y_test)})\n")
print(f"y_test: {y_test[0:5]}\n")
print(f"X_test (few rows): {X_test[0:5,:]}")  # 5 rows
print("\n")
print(f"+++ Training +++   Data used for modeling... (training data: {len(y_train)})\n")
print(f"y_train: {y_train[0:5]}\n")
print(f"X_train (few rows): {X_train[0:5,:]}")  # 5 rows ,:

training with 1414 rows;  testing with 354 rows

+++ Testing +++   Held-out data... (testing data: 354)

y_test: [1. 6. 2. 0. 2.]

X_test (few rows): [[ 0.  0.  3. 13. 10.  1.  0.  0.  0.  0.  3. 16. 16.  4.  0.  0.  0.  0.
   1. 16. 16.  2.  0.  0.  0.  0.  6. 16. 16.  1.  0.  0.  0.  0.  4. 16.
  16.  1.  0.  0.  0.  0.  4. 16. 16.  3.  0.  0.  0.  0.  7. 16. 16.  0.
   0.  0.  0.  0.  2. 14. 16.  5.  0.  0.]
 [ 0.  0.  2. 12.  1.  0.  0.  0.  0.  0. 11. 12.  0.  0.  0.  0.  0.  2.
  16.  4.  0.  0.  0.  0.  0.  6. 16. 10. 10.  5.  0.  0.  0.  5. 16. 15.
  12. 14.  6.  0.  0.  4. 16.  3.  0.  8. 12.  0.  0.  0. 14.  9.  4. 11.
  13.  0.  0.  0.  3. 14. 16. 12.  3.  0.]
 [ 0.  4. 16. 15.  2.  0.  0.  0.  0. 11. 15. 15.  7.  0.  0.  0.  0.  9.
  10.  6. 14.  0.  0.  0.  0.  0.  0.  7. 15.  0.  0.  0.  0.  0.  0. 13.
  10.  0.  0.  0.  0.  0.  1. 16.  7.  2.  2.  0.  0.  1. 12. 16. 15. 16.
  15.  0.  0.  4. 16. 16. 16. 12. 11.  0.]
 [ 0.  0.  1. 12. 12.  3.  0.  0.  0.  0.  9. 13.  5. 1

In [16]:
#
# for NNets, it's important to keep the feature values near 0, say -1. to 1. or so
#    This is done through the "StandardScaler" in scikit-learn
# 
from sklearn.preprocessing import StandardScaler

#
# do we want to use a Scaler?
#
USE_SCALER = True   # this variable is important! It tracks if we need to use the scaler...

# we "train the scaler"  (computes the mean and standard deviation)
if USE_SCALER == True:
    scaler = StandardScaler()
    scaler.fit(X_train)  # Scale with the training data! ave becomes 0; stdev becomes 1
else:
    # this one does no scaling!  We still create it to be consistent:
    scaler = StandardScaler(copy=True, with_mean=False, with_std=False) # no scaling
    scaler.fit(X_train)  # still need to fit, though it does not change...

scaler   # is now defined and ready to use...

# ++++++++++++++++++++++++++++++++++++++++++++++++++++++

# Here are our scaled training and testing sets:

X_train_scaled = scaler.transform(X_train) # scale!
X_test_scaled = scaler.transform(X_test) # scale!

y_train_scaled = y_train.copy()  # the predicted/desired labels are not scaled
y_test_scaled = y_test.copy()  # not using the scaler



In [29]:
#
# let's create a table for showing our data and its predictions...
#
def ascii_table(X,y,scaler_to_invert=None):
    """ print a table of inputs and outputs """
    np.set_printoptions(precision=2)  # Let's use less precision
    if scaler_to_invert == None:  # don't use the scaler
        X = X
    else:
        X = scaler_to_invert.inverse_transform(X)
    print(f"{'input ':>58s} -> {'pred':^7s} {'des':<5s}") 
    for i in range(len(y)):
        # whoa! serious f-string formatting:
        print(f"{str(X[i,0:64]):>58s} -> {'?':^7s} {str(y[i]):<21s}")   # !s is str ...
    print()
    
# to show the table with the scaled data:
ascii_table(X_train_scaled[0:5,:],y_train_scaled[0:5],None)

# to show the table with the original data:
ascii_table(X_train_scaled[0:5,:],y_train_scaled[0:5],scaler_to_invert=scaler)

                                                    input  ->  pred   des  
[ 0.   -0.33 -0.26  0.5  -0.19 -1.03 -0.41 -0.12 -0.07 -0.62 -1.54  1.02
  1.2  -0.86 -0.52 -0.12 -0.05 -0.72 -1.71  1.2   1.44 -0.46 -0.55 -0.12
 -0.04 -0.77 -1.45  1.22  0.98 -0.78 -0.63 -0.04  0.   -0.68 -1.04  0.94
  0.95 -1.15 -0.82  0.   -0.07 -0.53 -0.75  1.2   0.84 -1.44 -0.79 -0.1
 -0.04 -0.4  -0.61  1.24  0.31 -1.45 -0.76 -0.21 -0.03 -0.29 -0.12  0.89
  0.44 -0.98 -0.51 -0.2 ] ->    ?    1.0                  
[ 0.   -0.33  1.01  0.97 -1.56 -1.03 -0.41 -0.12 -0.07 -0.31 -0.06  0.52
  0.36 -1.35 -0.52 -0.12 -0.05 -0.72 -1.71  0.34  0.63 -1.26 -0.55 -0.12
 -0.04 -0.77 -1.13  0.37  0.48 -0.78 -0.63 -0.04  0.   -0.68  0.53  1.1
  0.95  1.25  1.19  0.   -0.07 -0.53 -0.59  1.36 -0.6  -0.56 -0.56 -0.1
 -0.04 -0.4  -0.08  0.67 -1.77 -1.45 -0.76 -0.21 -0.03 -0.29  1.45 -1.37
 -2.38 -1.15 -0.51 -0.2 ] ->    ?    7.0                  
[ 0.    0.83  1.43  0.97  0.5  -0.67 -0.41 -0.12 -0.07  1.56  0.86 -2.
  0.57 -

In [30]:
from sklearn.neural_network import MLPClassifier

#
# Here's where you can change the number of hidden layers
# and number of neurons!  It's in the tuple  hidden_layer_sizes:
#
nn_classifier = MLPClassifier(hidden_layer_sizes=(6,7),  
                    # hidden_layer_sizes=(6,7)   means   4 inputs -> 6 hidden -> 7 hidden -> 3 outputs
                    max_iter=500,      # how many times to train
                    # activation="tanh", # the "activation function" input -> output
                    # solver='sgd',      # the algorithm for optimizing weights
                    verbose=True,      # False to "mute" the training
                    shuffle=True,      # reshuffle the training epochs?
                    random_state=None, # set for reproduceability
                    learning_rate_init=.1,       # learning rate: the amt of error to backpropagate!
                    learning_rate = 'adaptive')  # soften feedback as it converges

# documentation:
# scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html 
#     Try other network sizes / other parameters ...

print("\n\n++++++++++  TRAINING:  begin  +++++++++++++++\n\n")
nn_classifier.fit(X_train_scaled, y_train_scaled)
print("\n++++++++++  TRAINING:   end  +++++++++++++++")
print(f"The analog prediction error (the loss) is {nn_classifier.loss_}")



++++++++++  TRAINING:  begin  +++++++++++++++


Iteration 1, loss = 1.73632514
Iteration 2, loss = 0.80833140
Iteration 3, loss = 0.56757770
Iteration 4, loss = 0.42453877
Iteration 5, loss = 0.35601718
Iteration 6, loss = 0.27280595
Iteration 7, loss = 0.23626621
Iteration 8, loss = 0.21876719
Iteration 9, loss = 0.22525131
Iteration 10, loss = 0.21896201
Iteration 11, loss = 0.25138663
Iteration 12, loss = 0.17578176
Iteration 13, loss = 0.20628053
Iteration 14, loss = 0.20092508
Iteration 15, loss = 0.25917876
Iteration 16, loss = 0.27091886
Iteration 17, loss = 0.43326867
Iteration 18, loss = 0.30550628
Iteration 19, loss = 0.39855622
Iteration 20, loss = 0.32356158
Iteration 21, loss = 0.25436701
Iteration 22, loss = 0.18455260
Iteration 23, loss = 0.14693288
Iteration 24, loss = 0.12322321
Iteration 25, loss = 0.09922518
Iteration 26, loss = 0.08080028
Iteration 27, loss = 0.06940429
Iteration 28, loss = 0.06276340
Iteration 29, loss = 0.05840606
Iteration 30, loss = 0.07208279

In [32]:
#
# how did it do on the testing data?
# 

SEE_PROBS = False

def ascii_table_for_classifier(Xsc,y,nn,scaler):
    """ a table including predictions using nn.predict """
    predictions = nn.predict(Xsc)            # all predictions
    prediction_probs = nn.predict_proba(Xsc) # all prediction probabilities
    Xpr = scaler.inverse_transform(Xsc)      # Xpr is the "X to print": the unscaled data
    # count correct
    num_correct = 0
    # printing
    print(f"{'input ':>28s} -> {'pred':^12s} {'des.':^12s}") 
    for i in range(len(y)):
        pred = predictions[i]
        pred_probs = str(prediction_probs[i,:])
        desired = y[i].astype(int)
        # print(pred, desired, pred_probs)
        pred_species = float(pred)
        des_species  = float(desired)
        if pred_species != des_species: result = "  incorrect: " + pred_probs
        else: result = "  correct" + (": "+pred_probs if SEE_PROBS else "") ; num_correct += 1
        # Xpr = Xsc  # if you want to see the scaled versions
        print(f"{Xpr[i,0:64]!s:>28s} -> {str(pred_species):^12s} {str(des_species):12s} {str(result):^10s}") 
    print(f"\ncorrect predictions: {num_correct} out of {len(y)}")
    
#
# let's see how it did on the test data (also the training data!)
#
ascii_table_for_classifier(X_test_scaled,
                           y_test_scaled,
                           nn_classifier,
                           scaler) 

                      input  ->     pred         des.    
[0.00e+00 0.00e+00 3.00e+00 1.30e+01 1.00e+01 1.00e+00 0.00e+00 0.00e+00
 8.67e-19 0.00e+00 3.00e+00 1.60e+01 1.60e+01 4.00e+00 0.00e+00 0.00e+00
 0.00e+00 0.00e+00 1.00e+00 1.60e+01 1.60e+01 2.00e+00 0.00e+00 0.00e+00
 0.00e+00 0.00e+00 6.00e+00 1.60e+01 1.60e+01 1.00e+00 0.00e+00 0.00e+00
 0.00e+00 0.00e+00 4.00e+00 1.60e+01 1.60e+01 1.00e+00 0.00e+00 0.00e+00
 0.00e+00 0.00e+00 4.00e+00 1.60e+01 1.60e+01 3.00e+00 0.00e+00 0.00e+00
 0.00e+00 0.00e+00 7.00e+00 1.60e+01 1.60e+01 0.00e+00 0.00e+00 0.00e+00
 0.00e+00 0.00e+00 2.00e+00 1.40e+01 1.60e+01 5.00e+00 0.00e+00 0.00e+00] ->     1.0      1.0            correct 
[ 0.00e+00  0.00e+00  2.00e+00  1.20e+01  1.00e+00 -8.88e-16  0.00e+00
  0.00e+00  8.67e-19  0.00e+00  1.10e+01  1.20e+01  0.00e+00  0.00e+00
  0.00e+00  0.00e+00  0.00e+00  2.00e+00  1.60e+01  4.00e+00  0.00e+00
  0.00e+00  0.00e+00  0.00e+00  0.00e+00  6.00e+00  1.60e+01  1.00e+01
  1.00e+01  5.00e+00  0.00e+00  0

In [33]:
#
# We don't usually look inside the NNet, but we can: it's open-box modeling...
#
if True:  # do we want to see all of the parameters?
    np.set_printoptions(precision=2)  # Let's use less precision
    nn = nn_classifier  # less to type?
    print("\n\n+++++ parameters, weights, etc. +++++\n")
    print(f"\nweights/coefficients:\n")
    for i, wts in enumerate(nn.coefs_):
        print(f"[[ Layer {i} ]]\n   has shape = {wts.shape} and weights =\n{wts}")
        print(f"   with intercepts:\n {nn.intercepts_[i]}\n")
    print()
    print(f"\nall parameters: {nn.get_params()}")



+++++ parameters, weights, etc. +++++


weights/coefficients:

[[ Layer 0 ]]
   has shape = (64, 6) and weights =
[[ 8.95e-02  3.95e-02  4.03e-04  1.02e-03 -1.24e-03 -5.14e-04]
 [ 2.19e-02  2.69e-01 -4.18e-01 -7.43e-01 -4.29e-01  2.91e-01]
 [ 2.07e-01 -6.17e-01 -4.40e-01 -1.55e+00  7.93e-03 -8.83e-01]
 [ 1.02e+00 -1.44e+00  2.73e-03  8.41e-01  2.05e-01 -2.27e-01]
 [ 1.29e+00 -8.44e-01 -3.20e-01 -1.26e+00  1.14e-01 -2.59e-01]
 [ 2.19e+00 -3.71e+00 -1.34e+00  5.19e-01 -1.86e-01 -6.91e-01]
 [ 1.42e-01 -1.52e+00 -4.92e-01 -9.42e-01 -7.87e-01 -1.34e+00]
 [-1.99e+00 -8.30e-01 -6.93e-01 -1.00e+00 -1.57e+00  2.45e-02]
 [ 2.47e-01 -5.59e-01 -6.42e-01 -1.37e-01 -6.57e-01  1.03e+00]
 [-5.56e-01 -2.00e+00  1.10e+00  2.96e-01 -1.08e-01 -4.75e-02]
 [ 9.64e-01 -9.04e-01  1.08e+00 -8.21e-01  6.98e-01 -2.07e-01]
 [ 1.39e+00 -5.35e-01  6.64e-01 -1.28e+00 -7.23e-01  2.19e+00]
 [ 1.08e+00 -1.63e+00 -1.82e+00  1.37e-01 -3.30e-01 -1.96e+00]
 [-4.73e-01 -5.03e-01  3.95e-01  2.05e+00  7.99e-01  7.09e-01]
 [

In [35]:
#
# final predictive model (random forests), with tuned parameters + ALL data incorporated
#


def predictive_model( Features, MODEL, SCALER ):
    """ input: a list of 64 features (pixel intensities 0-63)
        output: the predicted digit
    """
    our_features = np.asarray([Features])                 # extra brackets needed
    scaled_features = SCALER.transform(our_features)      # we have to scale the features into "scaled space"
    predicted_cat = MODEL.predict(scaled_features)        # then, the nnet can predict our "cat" variables
    prediction_probs = nn.predict_proba(scaled_features) # all prediction probabilities
    # our_features = SCALER.inverse_transform(scaled_features)  # we can convert back (optional!)
    predicted_species = str(predicted_cat[0])     # (it's extra-nested) get the species name
    return predicted_species, prediction_probs


LoD = [[0,0,0,8,14,0,0,0,0,0,5,16,11,0,0,0,0,1,15,14,1,6,0,0,0,7,16,5,3,16,8,0,0,8,16,8,14,16,2,0,0,0,6,14,16,11,0,0,0,0,0,6,16,4,0,0,0,0,0,10,15,0,0,0],
[0,0,0,5,14,12,2,0,0,0,7,15,8,14,4,0,0,0,6,2,3,13,1,0,0,0,0,1,13,4,0,0,0,0,1,11,9,0,0,0,0,8,16,13,0,0,0,0,0,5,14,16,11,2,0,0,0,0,0,6,12,13,3,0],
[0,0,0,3,16,3,0,0,0,0,0,12,16,2,0,0,0,0,8,16,16,4,0,0,0,7,16,15,16,12,11,0,0,8,16,16,16,13,3,0,0,0,0,7,14,1,0,0,0,0,0,6,16,0,0,0,0,0,0,4,14,0,0,0],
[0,0,0,3,15,10,1,0,0,0,0,11,10,16,4,0,0,0,0,12,1,15,6,0,0,0,0,3,4,15,4,0,0,0,0,6,15,6,0,0,0,4,15,16,9,0,0,0,0,0,13,16,15,9,3,0,0,0,0,4,9,14,7,0],
[0,0,0,3,16,3,0,0,0,0,0,10,16,11,0,0,0,0,4,16,16,8,0,0,0,2,14,12,16,5,0,0,0,10,16,14,16,16,11,0,0,5,12,13,16,8,3,0,0,0,0,2,15,3,0,0,0,0,0,4,12,0,0,0],
[0,0,7,15,15,4,0,0,0,8,16,16,16,4,0,0,0,8,15,8,16,4,0,0,0,0,0,10,15,0,0,0,0,0,1,15,9,0,0,0,0,0,6,16,2,0,0,0,0,0,8,16,8,11,9,0,0,0,9,16,16,12,3,0]]


SEE_PROBS = False

# run on each one:
for Features in LoD:
    MODEL = nn_classifier
    SCALER = scaler
    name, probs = predictive_model( Features, MODEL, SCALER )  # pass in the model, too!
    prob_str = "   with probs: " + str(probs) if SEE_PROBS == True else ""
    print(f"I predict {name:>12s} from the features {Features}  {prob_str}")    # Answers in the assignment...

I predict          4.0 from the features [0, 0, 0, 8, 14, 0, 0, 0, 0, 0, 5, 16, 11, 0, 0, 0, 0, 1, 15, 14, 1, 6, 0, 0, 0, 7, 16, 5, 3, 16, 8, 0, 0, 8, 16, 8, 14, 16, 2, 0, 0, 0, 6, 14, 16, 11, 0, 0, 0, 0, 0, 6, 16, 4, 0, 0, 0, 0, 0, 10, 15, 0, 0, 0]  
I predict          2.0 from the features [0, 0, 0, 5, 14, 12, 2, 0, 0, 0, 7, 15, 8, 14, 4, 0, 0, 0, 6, 2, 3, 13, 1, 0, 0, 0, 0, 1, 13, 4, 0, 0, 0, 0, 1, 11, 9, 0, 0, 0, 0, 8, 16, 13, 0, 0, 0, 0, 0, 5, 14, 16, 11, 2, 0, 0, 0, 0, 0, 6, 12, 13, 3, 0]  
I predict          4.0 from the features [0, 0, 0, 3, 16, 3, 0, 0, 0, 0, 0, 12, 16, 2, 0, 0, 0, 0, 8, 16, 16, 4, 0, 0, 0, 7, 16, 15, 16, 12, 11, 0, 0, 8, 16, 16, 16, 13, 3, 0, 0, 0, 0, 7, 14, 1, 0, 0, 0, 0, 0, 6, 16, 0, 0, 0, 0, 0, 0, 4, 14, 0, 0, 0]  
I predict          8.0 from the features [0, 0, 0, 3, 15, 10, 1, 0, 0, 0, 0, 11, 10, 16, 4, 0, 0, 0, 0, 12, 1, 15, 6, 0, 0, 0, 0, 3, 4, 15, 4, 0, 0, 0, 0, 6, 15, 6, 0, 0, 0, 4, 15, 16, 9, 0, 0, 0, 0, 0, 13, 16, 15, 9, 3, 0, 0, 0, 0, 4, 9, 14, 7,

### Second, from the iris example create a _pixel-predicting_ &nbsp; NNet

Choose a pixel to predict!
+ It _can_ be #42 -- or choose another one?!
+ This will be _regression_, not classification
+ It will show off NNets' ability to generate or "hallucinate" pixels/digits/images/etc.!
+ The _digit dreaming_ problem will extend this further...

<b>Predict-a-pixel</b> (regression)...
+ As the penultimate part of this digits-analysis with NNets, 
+ create a regressor that predicts pixel 42 from the other 63 pixels!
+ Remember that pixel 42 will be `A[:,42]`
+ and, the other 63, plus the digit-species, will be `np.concatenate((A[:,0:42], A[:,43:]),axis=1)`
+ see the iris_modeler for an example for the irises' botanical features...

In [37]:
print(A)
print(len(A))
print(len(A[0]))

[[ 0.  0.  9. ...  1.  0.  8.]
 [ 0.  0. 11. ...  0.  0.  9.]
 [ 0.  0.  1. ...  0.  0.  0.]
 ...
 [ 0.  0.  1. ...  0.  0.  8.]
 [ 0.  0.  2. ...  0.  0.  9.]
 [ 0.  0. 10. ...  1.  0.  8.]]
1768
65


In [46]:
#
# Here we set up for a regression model that will predict pixel intensity 42
#


print("+++ Start of data-assembly for feature-regression! +++\n")
# construct the correct X_all from the columns we want
# we use np.concatenate to combine parts of the dataset to get all-except-column 0:
#                     exclude 0  , include 1 to the end
X_all = np.concatenate( (A[:,0:42], A[:,43:]), axis=1)    # includes columns 1, 2, 3, and 4

# if we wanted all-except-column 1:   X_all = np.concatenate( (A[:,0:1], A[:,2:]),axis=1)  # columns 0, 2, 3, and 4
# if we wanted all-except-column 2:   X_all = np.concatenate( (A[:,0:2], A[:,3:]),axis=1)  # columns 0, 1, 3, and 4
# if we wanted all-except-column 3:   X_all = np.concatenate( (A[:,0:3], A[:,4:]),axis=1)  # columns 0, 1, 2, and 4
# if we wanted all-except-column 4:   X_all = np.concatenate( (A[:,0:4], A[:,5:]),axis=1)  # columns 0, 1, 2, and 3
# (slicing is forgiving)


y_all = A[:,42]                    # y (labels) ... is all of column 0, sepallen (sepal length) 
#                                 # change the line above to make other columns the target (y_all)
print(f"y_all is \n {y_all}")
print() 
print(f"X_all (the features, first few rows) is \n {X_all[:5,:]}")

+++ Start of data-assembly for feature-regression! +++

y_all is 
 [16.  0. 16. ... 16.  0. 16.]

X_all (the features, first few rows) is 
 [[ 0.  0.  9. 14.  8.  1.  0.  0.  0.  0. 12. 14. 14. 12.  0.  0.  0.  0.
   9. 10.  0. 15.  4.  0.  0.  0.  3. 16. 12. 14.  2.  0.  0.  0.  4. 16.
  16.  2.  0.  0.  0.  3.  8. 10. 13.  2.  0.  0.  1. 15.  1.  3. 16.  8.
   0.  0.  0. 11. 16. 15. 11.  1.  0.  8.]
 [ 0.  0. 11. 12.  0.  0.  0.  0.  0.  2. 16. 16. 16. 13.  0.  0.  0.  3.
  16. 12. 10. 14.  0.  0.  0.  1. 16.  1. 12. 15.  0.  0.  0.  0. 13. 16.
   9. 15.  2.  0.  0.  0.  3.  0.  9. 11.  0.  0.  0.  0.  0.  9. 15.  4.
   0.  0.  0.  9. 12. 13.  3.  0.  0.  9.]
 [ 0.  0.  1.  9. 15. 11.  0.  0.  0.  0. 11. 16.  8. 14.  6.  0.  0.  2.
  16. 10.  0.  9.  9.  0.  0.  1. 16.  4.  0.  8.  8.  0.  0.  4. 16.  4.
   0.  8.  8.  0.  0.  1.  5.  1. 11.  3.  0.  0.  0. 12. 12. 10. 10.  0.
   0.  0.  0.  1. 10. 13.  3.  0.  0.  0.]
 [ 0.  0.  0.  0. 14. 13.  1.  0.  0.  0.  0.  5. 16. 16.  2.  0.

In [47]:
#
# we scramble the data, to give a different TRAIN/TEST split each time...
# 
indices = np.random.permutation(len(y_all))  # indices is a permutation-list

# we scramble both X and y, necessarily with the same permutation
X_all = X_all[indices]              # we apply the _same_ permutation to each!
y_all = y_all[indices]              # again...
print("target values to predict: \n",y_all)
print("\nfeatures (a few)\n", X_all[0:5,:])

target values to predict: 
 [ 2.  0.  0. ... 11.  7. 11.]

features (a few)
 [[ 0.  1.  9. 12. 16. 16.  4.  0.  0.  1. 11.  8.  7. 16.  4.  0.  0.  0.
   0.  0.  8. 13.  0.  0.  0.  0.  5. 11. 15. 15.  9.  0.  0.  0. 16. 15.
  13.  5.  2.  0.  0.  0. 16.  5.  0.  0.  0.  0.  0.  9. 14.  1.  0.  0.
   0.  0.  0. 14. 10.  0.  0.  0.  0.  7.]
 [ 0.  2. 13. 16. 16. 15.  4.  0.  0.  7. 12.  8.  8. 16. 12.  0.  0.  0.
   0.  0.  8. 16.  7.  0.  0.  0.  0.  0. 14. 10.  0.  0.  0.  0.  0.  0.
  12. 15.  3.  0.  0.  0.  0.  2. 16. 11.  0.  0.  0.  4.  4.  7. 16. 10.
   0.  0.  2. 15. 16. 16. 12.  1.  0.  3.]
 [ 0.  0.  9. 15. 13.  3.  0.  0.  0.  2. 14.  2.  6.  5.  0.  0.  0.  4.
   9.  0.  0. 15.  4.  0.  0.  1. 13.  5.  7. 16.  6.  0.  0.  0.  2.  7.
   7. 14.  3.  0.  0.  0.  0.  1. 14.  1.  0.  0.  0.  3.  1. 10.  9.  0.
   0.  0.  0. 11. 15.  9.  1.  0.  0.  9.]
 [ 0.  0.  1.  9. 13.  1.  0.  0.  0.  1. 12. 14.  5.  0.  0.  0.  0.  2.
  16.  5.  0.  0.  0.  0.  0.  5. 15.  0.  3.  0.  0. 

In [48]:
#
# We next separate into test data and training data ... 
#    + We will train on the training data...
#    + We will _not_ look at the testing data to build the model
#
# Then, afterward, we will test on the testing data -- and see how well we do!
#

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2)

print(f"training with {len(y_train)} rows;  testing with {len(y_test)} rows\n" )

print(f"Held-out data... (testing data: {len(y_test)})")
print(f"y_test: {y_test}\n")
print(f"X_test (few rows): {X_test[0:5,:]}")  # 5 rows
print()
print(f"Data used for modeling... (training data: {len(y_train)})")
print(f"y_train: {y_train}\n")
print(f"X_train (few rows): {X_train[0:5,:]}")  # 5 rows

training with 1414 rows;  testing with 354 rows

Held-out data... (testing data: 354)
y_test: [12.  8. 16. 16. 15.  2.  4.  2. 15.  0. 15.  0.  0. 14.  0.  0.  9.  4.
  2.  8. 14.  5.  1. 12.  6.  0.  9. 14.  0.  5. 13.  0. 13.  3.  0. 11.
  9.  1.  0.  0.  3. 16. 16.  0. 13.  4. 16. 16.  0.  0.  0.  0. 13.  0.
  4. 13.  0.  0. 10.  4. 13.  0.  1. 15.  0.  1.  3.  0.  8.  8.  0.  9.
  0.  0. 16.  4. 16. 12.  4.  2.  0.  3.  0.  0. 12. 16. 16.  0. 12.  7.
 15.  0.  0. 16. 10. 14. 16.  6.  3.  0.  0.  0.  0.  0. 16. 15. 16. 13.
 16.  0.  0.  0. 16. 15.  0. 16.  0.  0.  0.  6.  0. 15. 13.  0. 13.  0.
  9.  0. 16. 16.  6.  9.  8. 11.  0.  0.  2.  0.  0.  0.  0. 16.  4. 16.
  8.  5.  2. 16.  0.  0.  0.  0.  0.  0. 16.  0.  0. 15.  0. 15. 13.  6.
 16. 15.  4.  0. 16. 13. 16.  0. 12.  5. 16. 11.  1. 16.  2. 15. 15.  8.
  7. 16.  3. 12. 16.  0.  0. 16.  5. 16. 11.  0.  0.  0. 15. 16.  0.  4.
 16.  3.  8. 16. 13. 16.  0. 14.  0.  0.  0.  0. 12. 16. 16. 15. 15. 16.
  0.  0.  0.  2.  0.  0. 16. 1

In [49]:
#
# for NNets, it's important to keep the feature values near 0, say -1. to 1. or so
#    This is done through the "StandardScaler" in scikit-learn
#
from sklearn.preprocessing import StandardScaler

USE_SCALER = True   # this variable is important! It tracks if we need to use the scaler...

# we "train the scaler"  (computes the mean and standard deviation)
if USE_SCALER == True:
    scaler = StandardScaler()
    scaler.fit(X_train)  # Scale with the training data! ave becomes 0; stdev becomes 1
else:
    # this one does no scaling!  We still create it to be consistent:
    scaler = StandardScaler(copy=True, with_mean=False, with_std=False) # no scaling
    scaler.fit(X_train)  # still need to fit, though it does not change...

scaler   # is now defined and ready to use...

# ++++++++++++++++++++++++++++++++++++++++++++++++++++++

# Here are our scaled training and testing sets:

X_train_scaled = scaler.transform(X_train) # scale!
X_test_scaled = scaler.transform(X_test) # scale!

y_train_scaled = y_train  # the predicted/desired labels are not scaled
y_test_scaled = y_test  # not using the scaler

# reused from above - seeing the scaled data 
ascii_table(X_train_scaled[0:5,:],y_train_scaled[0:5],None)

# reused from above - seeing the unscaled data (inverting the scaler)
ascii_table(X_train_scaled[0:5,:],y_train_scaled[0:5],scaler)

                                                    input  ->  pred   des  
[ 0.   -0.35 -0.68  0.26  0.96  0.4  -0.41 -0.12 -0.06 -0.63  0.47 -1.25
 -0.67  0.8  -0.51 -0.13 -0.04 -0.45 -0.49 -1.19 -0.01  0.83  0.06 -0.12
 -0.03 -0.78 -1.28  0.2   1.    1.44  2.93 -0.05  0.   -0.67 -0.26  0.62
  0.97 -1.14 -0.83  0.   -0.07 -0.53 -0.33  0.54 -1.45 -0.81 -0.09 -0.03
 -0.4  -1.35  0.47 -1.02 -1.47 -0.77 -0.21  0.   -0.31 -0.7   0.43 -2.43
 -1.16 -0.51 -0.19  0.88] ->    ?    1.0                  
[ 0.   -0.35  0.16  0.5  -2.48 -1.02 -0.41 -0.12 -0.06 -0.63  0.29 -0.49
 -2.12 -1.34 -0.51 -0.13 -0.04  0.1   1.09 -1.01 -1.14 -1.24 -0.55 -0.12
 -0.03  0.82  0.8  -1.49 -0.94 -0.93 -0.63 -0.05  0.    1.34  1.32  1.1
  0.97  1.25  0.58  0.   -0.07  0.78 -0.18 -1.22  0.13  1.69 -0.09 -0.03
 -0.4   1.15 -0.29 -0.83  0.68  1.04 -0.21  0.   -0.31 -0.11  0.2   0.85
  0.88 -0.26 -0.19  0.53] ->    ?    16.0                 
[ 0.   -0.35 -1.1  -2.79 -0.42  1.28  1.72 -0.12 -0.06 -0.63 -1.93 -1.75
  1.

In [50]:
#
# MLPRegressor predicts _floating-point_ outputs
#

from sklearn.neural_network import MLPRegressor

nn_regressor = MLPRegressor(hidden_layer_sizes=(6,7), 
                    max_iter=500,          # how many training epochs
                    verbose=True,          # do we want to watch as it trains?
                    shuffle=True,          # shuffle each epoch?
                    random_state=None,     # use for reproducibility
                    learning_rate_init=.1, # how much of each error to back-propagate
                    learning_rate = 'adaptive')  # how to handle the learning_rate

print("\n\n++++++++++  TRAINING:  begin  +++++++++++++++\n\n")
nn_regressor.fit(X_train_scaled, y_train_scaled)
print("++++++++++  TRAINING:   end  +++++++++++++++")
print(f"The (squared) prediction error (the loss) is {nn_regressor.loss_:<6.3f}")
print(f"And, its square root:         {nn_regressor.loss_ ** 0.5:<6.3f}")
print()



++++++++++  TRAINING:  begin  +++++++++++++++


Iteration 1, loss = 25.23087258
Iteration 2, loss = 9.20138222
Iteration 3, loss = 6.58969302
Iteration 4, loss = 4.75027988
Iteration 5, loss = 3.96322706
Iteration 6, loss = 3.38414919
Iteration 7, loss = 3.15149193
Iteration 8, loss = 2.97462945
Iteration 9, loss = 3.00268102
Iteration 10, loss = 2.81685207
Iteration 11, loss = 2.85016441
Iteration 12, loss = 2.68323319
Iteration 13, loss = 2.60539777
Iteration 14, loss = 2.64560795
Iteration 15, loss = 2.92725968
Iteration 16, loss = 2.90827434
Iteration 17, loss = 2.54317664
Iteration 18, loss = 2.27069494
Iteration 19, loss = 2.15969545
Iteration 20, loss = 2.31315383
Iteration 21, loss = 2.44827313
Iteration 22, loss = 2.18233541
Iteration 23, loss = 2.12327117
Iteration 24, loss = 2.27950706
Iteration 25, loss = 2.24948028
Iteration 26, loss = 1.97144555
Iteration 27, loss = 1.83820755
Iteration 28, loss = 1.92523111
Iteration 29, loss = 1.87996441
Iteration 30, loss = 1.7561457

In [51]:
#
# how did it do? Try out the TEST data...
#

def ascii_table_for_regressor(Xsc,y,nn,scaler):
    """ a table including predictions using nn.predict """
    predictions = nn.predict(Xsc) # all predictions
    Xpr = scaler.inverse_transform(Xsc)  # Xpr is the "X to print": unscaled data!
    # measure error
    error = 0.0
    # printing
    print(f"{'input ':>35s} ->  {'pred':^6s}  {'des.':^6s}   {'absdiff':^10s}") 
    for i in range(len(y)):
        pred = predictions[i]
        desired = y[i]
        result = abs(desired - pred)
        error += result
        # Xpr = Xsc   # if you'd like to see the scaled values
        print(f"{Xpr[i,:]!s:>35s} ->  {pred:<+6.2f}  {desired:<+6.2f}   {result:^10.2f}") 

    print("\n" + "+++++   +++++   +++++           ")
    print(f"average abs diff error:   {error/len(y):<6.3f}")
    print("+++++   +++++   +++++           ")
    
#
# let's see how it did on the test data (also the training data!)
#
ascii_table_for_regressor(X_test_scaled,
                          y_test_scaled,
                          nn_regressor,
                          scaler)   # this is our own f'n, above


                             input  ->   pred    des.     absdiff  
[ 0.00e+00  0.00e+00  0.00e+00  9.00e+00  1.50e+01  9.00e+00  0.00e+00
  0.00e+00  0.00e+00  0.00e+00  8.00e+00  1.50e+01  5.00e+00  1.20e+01
  2.00e+00  1.39e-17  0.00e+00  0.00e+00  1.50e+01  1.50e+01  3.00e+00
  1.30e+01  3.00e+00  0.00e+00  0.00e+00  0.00e+00  1.10e+01  1.60e+01
  1.60e+01  1.30e+01  0.00e+00  0.00e+00  0.00e+00  4.00e+00  1.60e+01
  1.00e+01  1.50e+01  0.00e+00  0.00e+00  0.00e+00  1.73e-18  3.00e+00
  0.00e+00  8.00e+00  7.00e+00  0.00e+00  3.47e-18  0.00e+00  0.00e+00
  1.20e+01  8.00e+00  1.00e+01  1.10e+01 -4.44e-16  0.00e+00  0.00e+00
  0.00e+00  8.88e-16  9.00e+00  1.30e+01  4.00e+00  0.00e+00  0.00e+00
  8.00e+00] ->  +17.47  +12.00      5.47   
[ 0.00e+00  0.00e+00  9.00e+00  9.00e+00  4.00e+00 -8.88e-16  0.00e+00
  0.00e+00  0.00e+00  0.00e+00  1.50e+01  1.50e+01  1.40e+01  1.20e+01
  0.00e+00  1.39e-17  0.00e+00  3.00e+00  1.00e+01  1.00e+00  0.00e+00
  1.20e+01  5.00e+00  0.00e+00  0.00

In [59]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor

def regressionPixelPredict(pix): 
    X_all = np.concatenate( (A[:,0:pix], A[:,pix+1:]), axis=1)    # includes columns excluding pix 
    y_all = A[:,pix] 
    indices = np.random.permutation(len(y_all))
    X_all = X_all[indices]                  
    y_all = y_all[indices]
    X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2)
    X_train_scaled = scaler.transform(X_train) # scale!
    X_test_scaled = scaler.transform(X_test) # scale!
    y_train_scaled = y_train  # the predicted/desired labels are not scaled
    y_test_scaled = y_test  # not using the scaler
    print("\n\n++++++++++  TRAINING:  begin  +++++++++++++++\n\n")
    nn_regressor.fit(X_train_scaled, y_train_scaled)
    print("++++++++++  TRAINING:   end  +++++++++++++++")
    print(f"The (squared) prediction error (the loss) is {nn_regressor.loss_:<6.3f}")
    print(f"And, its square root:         {nn_regressor.loss_ ** 0.5:<6.3f}")
    print()
    ascii_table_for_regressor(X_test_scaled,
                          y_test_scaled,
                          nn_regressor,
                          scaler)   # this is our own f'n, above
    
regressionPixelPredict(42)





++++++++++  TRAINING:  begin  +++++++++++++++


Iteration 1, loss = 18.59783049
Iteration 2, loss = 6.72888337
Iteration 3, loss = 4.80290038
Iteration 4, loss = 3.68686254
Iteration 5, loss = 3.32272617
Iteration 6, loss = 3.09059493
Iteration 7, loss = 2.80183059
Iteration 8, loss = 2.71605638
Iteration 9, loss = 2.55604850
Iteration 10, loss = 2.44052208
Iteration 11, loss = 2.60939175
Iteration 12, loss = 2.69338155
Iteration 13, loss = 2.88150043
Iteration 14, loss = 2.81414233
Iteration 15, loss = 2.49497462
Iteration 16, loss = 2.46114570
Iteration 17, loss = 2.35029964
Iteration 18, loss = 2.44993305
Iteration 19, loss = 2.83670167
Iteration 20, loss = 2.18145308
Iteration 21, loss = 2.56129730
Iteration 22, loss = 2.40427350
Iteration 23, loss = 2.30771215
Iteration 24, loss = 2.21058325
Iteration 25, loss = 2.19875668
Iteration 26, loss = 2.44416477
Iteration 27, loss = 2.33047136
Iteration 28, loss = 1.95803991
Iteration 29, loss = 1.99892098
Iteration 30, loss = 2.1091654